In [1]:
import scda
import pprint

# Demo 1: IrisAO APLC

## Set some design parameters

In [2]:
pupil_params = {'N': 200}
fpm_params = {'rad': 9.898/2, 'M':50}
ls_params = {}
image_params = {'c': 10., 'iwa':3.5, 'owa':10.}
design_params = {'Pupil': pupil_params, 'FPM': fpm_params, 'LS': ls_params, 'Image': image_params}

## file organization

In [3]:
test_dir = "test_aplc_wrapper" # a nominal destination for new AMPL programs
aux_dir = "~/SCDA/2d AMPL script - half pupil" # location of input masks for AMPL program

fileorg = {'work dir': test_dir, 'TelAp dir': aux_dir, 'FPM dir': aux_dir, 'LS dir': aux_dir, \
           'TelAp fname': "IRISAO_N=0200_center_half_spiders3=01_gapID=10_BW.dat", \
           'FPM fname': "CircPupil_N=0050_obs=00_center_half.dat", \
           'LS fname': "IRISAO-0_N=0200_center_half_spiders3=02_ID=20_OD=098.dat"}

## Options for constraints and optimizer

In [4]:
solver_params = {'method': 'bar', 'presolve': False, 'threads': 8}

## Create a coronagraph design object, then write its AMPL source code file.

In [5]:
irisao_coron = scda.HalfplaneAPLC( fileorg=fileorg, design=design_params, solver=solver_params )
irisao_coron.write_ampl(ampl_src_fname="test_aplc.mod", overwrite=True)

# Demo 2: SCDA paramater survey 

## Define a list of pupil parameters to survey
Parameters selected here are the primary mirror (pm), support struts (ss), secondary strut thickness (sst), secondary obstruction (so).
The last parameter, 'N' = pupil array diameter, has only one value and therefore is fixed for all designs. 

In [6]:
jpl_telap_all = {'Pupil': { 'pm': ['hex1', 'hex2', 'hex3', 'key24', 'pie12', 'pie08'],
                            'ss': ['Y60d','Yoff60d','X','Cross','T','Y90d'],
                            'sst': ['025','100'],
                            'sm': [True, False],
                            'N': 250 },
                 'LS': { 'shape':['ann', 'hex'], 'obscure':2, 'ppad':3, 'spad':4 } }

In [7]:
survey_ampl_dir = "./ampl_survey_test/" # where to write the AMPL source code

## Initiate a design parameter survey object with the above parameter combinations 

In [8]:
telap_survey = scda.DesignParamSurvey(scda.HalfplaneAPLC, jpl_telap_all, fileorg={'ampl src dir':survey_ampl_dir})
print("This survey has {0:d} design parameter combinations.".format(len(telap_survey.coron_list)))
print("{0:d} parameters are varied: {1}".format(len(telap_survey.varied_param_index), telap_survey.varied_param_index))

This survey has 288 design parameter combinations.
5 parameters are varied: (('Pupil', 'pm'), ('Pupil', 'ss'), ('Pupil', 'sst'), ('Pupil', 'sm'), ('LS', 'shape'))


## Print some attributes of an individual coronagraph object

In [9]:
i = 147
print("Telescope aperture file for design #{:d}: {:s}".format(i+1, telap_survey.coron_list[i].fileorg['TelAp fname']))
print("Focal plane mask file for design #{:d}: {:s}".format(i+1, telap_survey.coron_list[i].fileorg['FPM fname']))
print("Lyot stop file for design #{:d}: {:s}".format(i+1, telap_survey.coron_list[i].fileorg['LS fname']))

Telescope aperture file for design #148: /Users/ntz/SCDA/scda_pytools/TelAp_half_key24Y60d025sm0_N0250.dat
Focal plane mask file for design #148: /Users/ntz/SCDA/scda_pytools/FPM_half_occspot_M050.dat
Lyot stop file for design #148: /Users/ntz/SCDA/scda_pytools/LS_half_hex20D90_key24Pad03Y60d025sm0Pad04_N0250.dat


## List the varying parameter combinations

In [10]:
pprint.pprint(telap_survey.varied_param_combos)

(('hex1', 'Y60d', '025', True, 'ann'),
 ('hex1', 'Y60d', '025', True, 'hex'),
 ('hex1', 'Y60d', '025', False, 'ann'),
 ('hex1', 'Y60d', '025', False, 'hex'),
 ('hex1', 'Y60d', '100', True, 'ann'),
 ('hex1', 'Y60d', '100', True, 'hex'),
 ('hex1', 'Y60d', '100', False, 'ann'),
 ('hex1', 'Y60d', '100', False, 'hex'),
 ('hex1', 'Yoff60d', '025', True, 'ann'),
 ('hex1', 'Yoff60d', '025', True, 'hex'),
 ('hex1', 'Yoff60d', '025', False, 'ann'),
 ('hex1', 'Yoff60d', '025', False, 'hex'),
 ('hex1', 'Yoff60d', '100', True, 'ann'),
 ('hex1', 'Yoff60d', '100', True, 'hex'),
 ('hex1', 'Yoff60d', '100', False, 'ann'),
 ('hex1', 'Yoff60d', '100', False, 'hex'),
 ('hex1', 'X', '025', True, 'ann'),
 ('hex1', 'X', '025', True, 'hex'),
 ('hex1', 'X', '025', False, 'ann'),
 ('hex1', 'X', '025', False, 'hex'),
 ('hex1', 'X', '100', True, 'ann'),
 ('hex1', 'X', '100', True, 'hex'),
 ('hex1', 'X', '100', False, 'ann'),
 ('hex1', 'X', '100', False, 'hex'),
 ('hex1', 'Cross', '025', True, 'ann'),
 ('hex1', 'C

## Write the batch of AMPL files

In [11]:
telap_survey.write_ampl_batch(override_infile_status=True, overwrite=True)